# **Problem 2**

 **Installing Required dependencies**

In [1]:
!pip install transformers datasets tokenizers seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00


In [2]:
! pip install  accelerate -U
! pip install  transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.4 MB/s eta 0:00:00


 **Importing required libraries**

In [3]:
import datasets
import numpy as np
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy

In [4]:
from transformers import pipeline

In [5]:
import accelerate
import transformers
import json

Defining label2id for manual_output

In [6]:
label2id_manual_output={'O': '0',
 'B-PER': '1',
 'I-PER': '2',
 'B-ORG': '3',
 'I-ORG': '4',
 'B-LOC': '5',
 'I-LOC': '6',
 'B-MISC': '7',
 'I-MISC': '8'}

Defining label2id for chatgpt_output

In [7]:
label2id_chatgpt_output = {
    'O': '0',
    'B-PER': '1',
    'I-PER': '2',
    'B-ORG': '3',
    'I-ORG': '4',
    'B-LOC': '5',
    'I-LOC': '6',
    'B-GPE': '7',
    'I-GPE': '8',
    'B-DATE': '9',
    'I-DATE': '10',
    'B-TIME': '11',
    'I-TIME': '12',
    'B-MONEY': '13',
    'I-MONEY': '14',
    'B-PERCENT': '15',
    'I-PERCENT': '16',
    'B-FAC': '17',
    'I-FAC': '18',
    'B-PROD': '19',
    'I-PROD': '20',
    'B-EVENT': '21',
    'I-EVENT': '22',
    'B-WORK_OF_ART': '23',
    'I-WORK_OF_ART': '24',
    'B-LAW': '25',
    'I-LAW': '26',
    'B-LANGUAGE': '27',
    'I-LANGUAGE': '28'
}


In [8]:
#processing manualoutputs and chatgpt outputs to convert ners to nertags
def process_file(file_path, label2id):
    tokens=[]
    ner_tags=[]
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            json_obj = json.loads(line)
            tokens.append(json_obj['words'])
            ners=[]
            for i in json_obj['ner']:
                ners.append(label2id[i])
            ner_tags.append(ners)

    directory={}
    directory['tokens']=tokens
    directory['ner_tags']=ner_tags
    return directory

In [9]:
def tokenize_and_align_labels_for_our_outputs(examples,tokenizer,label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    dictioray_of_input_idsandlabels={}
    dictioray_of_input_idsandlabels['input_ids']=tokenized_inputs["input_ids"]
    dictioray_of_input_idsandlabels['labels']=labels
    return dictioray_of_input_idsandlabels

In [10]:
def decode_tokenized_datasets_ner(tokenized_datasets, tokenizer):
    manual_outputs=[]
    for item in range(len(tokenized_datasets['input_ids'])):
        temp=[]
        for tokenid in range(len(tokenized_datasets['input_ids'][item])):
            if(tokenized_datasets['input_ids'][item][tokenid]==101 or tokenized_datasets['input_ids'][item][tokenid]==102 ):
                continue
            else:
                pair=[]
                pair.append(tokenizer.decode(tokenized_datasets['input_ids'][item][tokenid]))
                pair.append(tokenized_datasets['labels'][item][tokenid])
            temp.append(pair)
        manual_outputs.append(temp[:-1])
    return manual_outputs

In [11]:
def convert_outputs_to_desired_format_ner(file_path,label2id,tokenizer):
  result=result = process_file(file_path, label2id)
  tokenized_datasets=tokenize_and_align_labels_for_our_outputs(result, tokenizer)
  outputs_in_wanted_format=decode_tokenized_datasets_ner(tokenized_datasets, tokenizer)
  return outputs_in_wanted_format


In [12]:
def tokenize_and_align_labels(examples,label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [13]:
from sklearn.metrics import f1_score
def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)
    return {
   "precision": results["overall_precision"],
   "recall": results["overall_recall"],
   "macro_f1": results["overall_f1"],
  "accuracy": results["overall_accuracy"],
  }

In [14]:
def process_ner_result(tokenizer, nlp, label2id, example):
    tokens = tokenizer.tokenize(example)
    ner_results = nlp(example)
    output_dict = []
    l = []
    for i in ner_results:
        pair = []
        pair.append(i['word'])
        pair.append(label2id[i['entity']])
        l.append(pair)

    i = 0
    j = 0
    while j < len(tokens) and i < len(l):
        if l[i][0] == tokens[j]:
            pair = []
            pair.append(l[i][0])
            pair.append(l[i][1])
            output_dict.append(pair)
            i += 1
        else:
            pair = []
            pair.append(tokens[j])
            pair.append(0)
            output_dict.append(pair)
        j += 1

    while j < len(tokens):
        pair = []
        pair.append(tokens[j])
        pair.append(0)
        output_dict.append(pair)
        j += 1

    return output_dict[:-1]

In [15]:
def getting_outputs_from_our_model(file_path,tokenizer, nlp, label2id):
  results_from_our_model=[]
  with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
      results_from_our_model.append(process_ner_result(tokenizer, nlp, label2id, line))
  return results_from_our_model

In [16]:
def flatten_list(list_of_lists):
    flattened_list = [item for sublist in list_of_lists for item in sublist]
    outputlist=[item[1] for item in flattened_list]
    return outputlist

**Defining training arguments with 5 epochs**

In [18]:
ner_args = TrainingArguments(
    "test-ner",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="no"
)

**Loading datasets and splitting them in to 70% training, 10% validation,and 20% test sets**

In [19]:
from datasets import load_dataset
lang='te'
raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

num_samples = len(raw_datasets['train']) // 5
subset_dataset = raw_datasets['train'].shuffle(seed=42).select(range(num_samples))
train_size = int(0.7 * num_samples)
val_size = int(0.1 * num_samples)
test_size = num_samples - train_size - val_size

train_data = subset_dataset.select(range(train_size))
val_data = subset_dataset.select(range(train_size, train_size + val_size))
test_data = subset_dataset.select(range(train_size + val_size, num_samples))

# Now you have 70% training, 10% validation, and 20% test data
raw_datasets['train']=train_data
raw_datasets['validation']=val_data
raw_datasets['test']=test_data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/507741 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/847 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2700 [00:00<?, ? examples/s]

In [20]:
label_list = raw_datasets["train"].features["ner_tags"].feature.names

In [21]:
from seqeval.metrics import classification_report
from sklearn.metrics import f1_score
metric = datasets.load_metric("seqeval")

<ipython-input-21-e3226eb5bfbf>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [22]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

## **INDIC-NER**

In [23]:
tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indicNER')

tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [24]:
tokenized_datasets = raw_datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/71083 [00:00<?, ? examples/s]

Map:   0%|          | 0/20311 [00:00<?, ? examples/s]

Map:   0%|          | 0/10154 [00:00<?, ? examples/s]

In [25]:
model = AutoModelForTokenClassification.from_pretrained("ai4bharat/indicNER", num_labels=7)

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [26]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [27]:
trainer = Trainer(
    model,
    ner_args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["validation"],
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics,
)

In [28]:
trainer.train()

TrainOutput(global_step=22215, training_loss=0.18105404862621308, metrics={'train_runtime': 5642.7598, 'train_samples_per_second': 62.986, 'train_steps_per_second': 3.937, 'total_flos': 1.3194452596744188e+16, 'train_loss': 0.18105404862621308, 'epoch': 5.0})

Test set macro f1 score in trained ner

In [29]:
predictions, labels, ner_metrics = trainer.predict(tokenized_datasets["test"])
print(ner_metrics['test_macro_f1'])

0.8235912888645391


Train set macro f1 score in trained ner

In [30]:
predictions, labels, ner_metrics = trainer.predict(tokenized_datasets["train"])
print(ner_metrics['test_macro_f1'])

0.9238869020174707


Validation set macro f1 score in trained ner

In [31]:
predictions, labels, ner_metrics = trainer.predict(tokenized_datasets["validation"])
print(ner_metrics['test_macro_f1'])

0.823618323742483


In [32]:
model.save_pretrained("ner_model")
tokenizer.save_pretrained("ner_tokenizer")
ner_tokenizer_fine_tuned=tokenizer

In [33]:
config = json.load(open("ner_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("ner_model/config.json","w"))
ner_model_fine_tuned = AutoModelForTokenClassification.from_pretrained("ner_model")
nlp_ner = pipeline("ner", model=ner_model_fine_tuned, tokenizer=ner_tokenizer_fine_tuned)

In [35]:
my_ner_model_outputs=getting_outputs_from_our_model('25sentences.txt',tokenizer, nlp_ner, label2id)

In [36]:
my_manual_outputs_ner=convert_outputs_to_desired_format_ner('manual_outputs.txt',label2id_manual_output,tokenizer)

In [38]:
chatgpt_outputs_ner=convert_outputs_to_desired_format_ner('chatgpt_outputs.txt',label2id_chatgpt_output,tokenizer)

In [39]:
my_ner_model_outputs=flatten_list(my_ner_model_outputs)
my_manual_outputs_ner=flatten_list(my_manual_outputs_ner)
chatgpt_outputs_ner=flatten_list(chatgpt_outputs_ner)

## **Comparision of both models**

Bert model macro-f1 score on test dataset:0.735874195424947

Ner model macro-f1 score on test dataset:0.8235912888645391

Indic-ner gives best precision than Indic-bert


# **Problem 4**

In [40]:
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_and_print_metrics(ground_truth, predicted):
    precision = precision_score(ground_truth, predicted, average='macro')
    recall = recall_score(ground_truth, predicted, average='macro')
    f1 = f1_score(ground_truth, predicted, average='macro')

    print("Precision:", precision)
    print("Recall:", recall)
    print("Macro F1 Score:", f1)
    print("\n")


Question 2 vs Question1(NER)

In [41]:
my_ner_model_outputs = [str(i) for i in my_ner_model_outputs]
calculate_and_print_metrics(my_manual_outputs_ner, my_ner_model_outputs)

Precision: 0.49239244761366735
Recall: 0.5052606522410369
Macro F1 Score: 0.4816845521179887




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Question 3 vs Question1(NER)

In [42]:
calculate_and_print_metrics(my_manual_outputs_ner, chatgpt_outputs_ner)

Precision: 0.5517292358119874
Recall: 0.4669581777004659
Macro F1 Score: 0.49964908729555996




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **PROBLEM 5**

#### **OBSERVATIONS**
The fine tuned bert model and fine tuned ner model and chatgpt are giving almost same macro-f1 scores.

In a way ner model gives some more good predictions than other two.

But only half of the predictions are right.

Ner model takes more time than Bert model while training



Hyperparameters are crucial settings that govern how a machine learning model learns. Tuning them can significantly impact a model's performance. Here's a report on the hyperparameters you've tuned for your BERT model and the optimal values chosen:

**Evaluation Strategy:** evaluation_strategy="epoch" means that the model evaluates performance after each training epoch. This is helpful for monitoring progress and early stopping.

**Learning Rate:** learning_rate=2e-5 determines the step size at each iteration during training. It's crucial for controlling the rate at which the model learns. A lower learning rate can lead to more stable training but might require more epochs.

**Batch Size:** per_device_train_batch_size=16 and per_device_eval_batch_size=16 specify the number of training and evaluation samples processed in each forward/backward pass. Smaller batch sizes can lead to more noise in the gradient but might generalize better.

**Number of Epochs:** num_train_epochs=5 defines the number of times the model goes through the entire training dataset. More epochs can lead to better convergence, but too many epochs can lead to overfitting.

**Weight Decay:** weight_decay=0.01 is a regularization term added to the loss function to prevent overfitting by penalizing large weights. It helps in controlling the complexity of the model.

**Save Strategy:** save_strategy="no" specifies not to save any checkpoints during training. This can save disk space but prevents the model from being saved for future use.

**Optimal Values Chosen:**

**Evaluation Strategy:** "epoch" for better monitoring and early stopping.

**Learning Rate:** 2e-5 for a balance between fast convergence and stability.

**Batch Size:** 16 for a moderate balance between speed and generalization.

**Number of Epochs:** 5 for a reasonable number of training iterations.

**Weight Decay:** 0.01 for regularization and controlling model complexity.

**Save Strategy:** "no" to save disk space.